In [203]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import seaborn as sns
from pylab import rcParams
import pandas as pd
from skimage import io

import shutil


%matplotlib inline

%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

rcParams['figure.figsize'] = 16, 10

np.random.seed(42)


In [204]:
sequences = ["0000"]

In [205]:
root_dir = Path().cwd().parent

data_dir = root_dir / "data"
(data_dir/"interim").mkdir(parents=True, exist_ok=True)
data_dir = root_dir / "data"
(data_dir/"processed").mkdir(parents=True, exist_ok=True)

proc_kitti_dir = data_dir/"processed/KITTI"
shutil.rmtree(proc_kitti_dir, ignore_errors=True)
(proc_kitti_dir / "images").mkdir(parents=True, exist_ok=True)
(proc_kitti_dir / "labels").mkdir(parents=True, exist_ok=True)

raw_image_dir = data_dir / "raw/KITTI/data_tracking_image_2/training/label_02/"
raw_label_dir = data_dir / "raw/KITTI/data_tracking_label_2/training/label_02/"


In [206]:
data_names = [
    "frame",
    "track_id",
    "type",
    "truncated",
    "occluded",
    "alpha",
    "bbox_left",
    "bbox_top",
    "bbox_right",
    "bbox_bottom",
    "dimension_width",
    "dimension_height",
    "dimension_length",
    "location_x",
    "location_y",
    "location_z",
    "rotation_y",
]

In [207]:
categories = ["Car", "Pedestrian", "Cyclist"]

In [208]:
sequence = sequences[0]
    
df = pd.read_csv(raw_label_dir / f"{sequence}.txt", delimiter=" ", names=data_names, index_col=False)
df.head()

,frame,track_id,type,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,dimension_width,dimension_height,dimension_length,location_x,location_y,location_z,rotation_y
0,0,-1,DontCare,-1,-1,-10.000000,219.310000,188.490000,245.500000,218.560000,-1000.000000,-1000.000000,-1000.000000,-10.000000,-1.000000,-1.000000,-1.000000
1,0,-1,DontCare,-1,-1,-10.000000,47.560000,195.280000,115.480000,221.480000,-1000.000000,-1000.000000,-1000.000000,-10.000000,-1.000000,-1.000000,-1.000000
2,0,0,Van,0,0,-1.793451,296.744956,161.752147,455.226042,292.372804,2.000000,1.823255,4.433886,-4.552284,1.858523,13.410495,-2.115488
3,0,1,Cyclist,0,0,-1.936993,737.619499,161.531951,931.112229,374.000000,1.739063,0.824591,1.785241,1.640400,1.675660,5.776261,-1.675458
4,0,2,Pedestrian,0,0,-2.523309,1106.137292,166.576807,1204.470628,323.876144,1.714062,0.767881,0.972283,6.301919,1.652419,8.455685,-1.900245


In [209]:
df1 = df[df["type"].isin(categories)]
df1.head()


,frame,track_id,type,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,dimension_width,dimension_height,dimension_length,location_x,location_y,location_z,rotation_y
3,0,1,Cyclist,0,0,-1.936993,737.619499,161.531951,931.112229,374.000000,1.739063,0.824591,1.785241,1.640400,1.675660,5.776261,-1.675458
4,0,2,Pedestrian,0,0,-2.523309,1106.137292,166.576807,1204.470628,323.876144,1.714062,0.767881,0.972283,6.301919,1.652419,8.455685,-1.900245
8,1,1,Cyclist,0,0,-1.935205,745.017137,156.393157,938.839722,374.000000,1.739063,0.824591,1.785241,1.700640,1.640419,5.778596,-1.664456
9,1,2,Pedestrian,0,0,-2.530402,1138.342096,160.872449,1223.338201,324.146788,1.714062,0.767881,0.972283,6.352093,1.593046,8.156156,-1.886840
13,2,1,Cyclist,0,0,-1.933364,752.406083,151.248515,946.562490,374.000000,1.739063,0.824591,1.785241,1.760880,1.605178,5.780931,-1.653453


In [210]:
df_temp = df1[df1["frame"].isin([int(0)])]
df_temp

,frame,track_id,type,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,dimension_width,dimension_height,dimension_length,location_x,location_y,location_z,rotation_y
3,0,1,Cyclist,0,0,-1.936993,737.619499,161.531951,931.112229,374.000000,1.739063,0.824591,1.785241,1.640400,1.675660,5.776261,-1.675458
4,0,2,Pedestrian,0,0,-2.523309,1106.137292,166.576807,1204.470628,323.876144,1.714062,0.767881,0.972283,6.301919,1.652419,8.455685,-1.900245


In [211]:
img_names = (raw_image_dir / sequence).glob("*.png")
for img_id, img_name in enumerate(tqdm(img_names)):
    save_name = f"{sequence}_{img_name.stem}"
    img_path = raw_image_dir / sequence / img_name
    img = io.imread(img_path)

    img_height = img.shape[0]
    img_width = img.shape[1]
    io.imsave(proc_kitti_dir / "images" / f"{save_name}.jpeg", img)

    df_temp = df1[df1["frame"].isin([int(img_name.stem)])]

    with (proc_kitti_dir / "labels" / f"{save_name}.txt").open(mode="w") as label_file:
        for index, row in df_temp.iterrows():
            category_idx = categories.index(row["type"])

            x1 = row["bbox_left"] / img_width
            y1 = row["bbox_top"] / img_height
            x2 = row["bbox_right"] / img_width
            y2 = row["bbox_bottom"] / img_height

            bbox_width = x2 - x1

            bbox_height = y2 - y1

            label_file.write(
                f"{category_idx} {x1 + bbox_width / 2} {y1 + bbox_height / 2} {bbox_width} {bbox_height}\n"
            )


154it [00:02, 59.61it/s]
